In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-08-12 14:16:30--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24,88M  10,3MB/s    in 2,4s    

2022-08-12 14:16:33 (10,3 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'str', 'isbn': 'str', 'rating': 'float32'})

In [4]:
df_books.isnull().sum()

isbn      0
title     0
author    1
dtype: int64

In [5]:
df_ratings.isnull().sum()

user      0
isbn      0
rating    0
dtype: int64

In [6]:
df_books.dropna(inplace=True)

In [7]:
df_books.isnull().sum()

isbn      0
title     0
author    0
dtype: int64

In [8]:
ratings = df_ratings['user'].value_counts()
df_ratings_rm = df_ratings[
                           ~df_ratings['user'].isin(ratings[ratings < 200].index)
]

In [9]:
ratings = df_ratings['isbn'].value_counts()
df_ratings_rm = df_ratings_rm[
                              ~df_ratings_rm['isbn'].isin(ratings[ratings < 100].index)
]

In [10]:
df = df_ratings_rm.pivot_table(index=['user'], columns=['isbn'], values='rating').fillna(0).T

In [11]:
df.index = df.join(df_books.set_index('isbn'))['title']

In [12]:
model = NearestNeighbors(metric='cosine')
model.fit(df.values)

NearestNeighbors(metric='cosine')

In [13]:
df.iloc[0].shape
title = 'The Queen of the Damned (Vampire Chronicles (Paperback))'
df.loc[title].shape
distance, indice = model.kneighbors([df.loc[title].values], n_neighbors = 6)
print(distance)
print(indice)
df.iloc[indice[0]].index.values
pd.DataFrame({
    'title' : df.iloc[indice[0]].index.values,
    'distance' : distance[0]
})\
.sort_values(by='distance', ascending=False)

[[0.         0.51784116 0.53763384 0.73450685 0.74486566 0.7939835 ]]
[[137 127 153 128 152 644]]


,title,distance
5,Catch 22,0.793984
4,The Witching Hour (Lives of the Mayfair Witches),0.744866
3,Interview with the Vampire,0.734507
2,The Tale of the Body Thief (Vampire Chronicles...,0.537634
1,"The Vampire Lestat (Vampire Chronicles, Book II)",0.517841
0,The Queen of the Damned (Vampire Chronicles (P...,0.000000


In [42]:
pd.DataFrame({
    'title' : df.iloc[indice[0]].index.values,
    'distance' : distance[0]
 }).sort_values(by='distance', ascending=False)

,title,distance
5,Catch 22,0.793984
4,The Witching Hour (Lives of the Mayfair Witches),0.744866
3,Interview with the Vampire,0.734507
2,The Tale of the Body Thief (Vampire Chronicles...,0.537634
1,"The Vampire Lestat (Vampire Chronicles, Book II)",0.517841
0,The Queen of the Damned (Vampire Chronicles (P...,0.000000


In [53]:
# function to return recommended books - this will be tested
def get_recommends(title = ""):
  try:
    book = df.loc[title]
  except KeyError as e:
    print('Este livro:', e, ', não existe')
    return
  distance, indice = model.kneighbors([book.values], n_neighbors=6)
  recommended_books = pd.DataFrame({
      'title' : df.iloc[indice[0]].index.values,
      'distance' : distance[0]
  }) \
  .sort_values(by='distance', ascending=False)\
  .head(5).values

  return [title, recommended_books]

In [54]:
get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")

['The Queen of the Damned (Vampire Chronicles (Paperback))',
 array([['Catch 22', 0.793983519077301],
        ['The Witching Hour (Lives of the Mayfair Witches)',
         0.7448656558990479],
        ['Interview with the Vampire', 0.7345068454742432],
        ['The Tale of the Body Thief (Vampire Chronicles (Paperback))',
         0.5376338362693787],
        ['The Vampire Lestat (Vampire Chronicles, Book II)',
         0.5178411602973938]], dtype=object)]

In [55]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You havn't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", array([["I'll Be Seeing You", 0.8016210794448853],
       ['The Weight of Water', 0.7708583474159241],
       ['The Surgeon', 0.7699410915374756],
       ['I Know This Much Is True', 0.7677075266838074],
       ['The Lovely Bones: A Novel', 0.7234864234924316]], dtype=object)]
You passed the challenge! 🎉🎉🎉🎉🎉
